In [14]:
import re

import pandas as pd


# Функция для извлечения размера (например, "60x85 см" или "60х85 см")
def extract_size(description):
    pattern = re.compile(r'(\d+)\s*[xхX×]\s*(\d+)\s*(см|cm)', re.IGNORECASE)
    match = pattern.search(description)
    if match:
        width = int(match.group(1))
        height = int(match.group(2))
        return (width, height)
    return None

# Функция для извлечения масштаба (например, "1:15 000 000")
def extract_scale(description):
    pattern = re.compile(r'1\s*:\s*([\d\s]+)')
    match = pattern.search(description)
    if match:
        scale_str = match.group(1).replace(" ", "")
        try:
            scale_value = int(scale_str)
            return scale_value
        except:
            return None
    return None

# Функция для извлечения года (актуальность данных)
def extract_year(description):
    pattern = re.compile(r'\b(19|20)\d{2}\b')
    match = pattern.search(description)
    if match:
        return int(match.group(0))
    return None

# Общая функция для поиска ключевых слов из списка
def extract_keywords(description, keywords):
    found = []
    description_lower = description.lower()
    for kw in keywords:
        if kw.lower() in description_lower:
            found.append(kw.lower())
    return found

# Функция для извлечения всех характеристик из описания
def extract_features(description):
    features = {}
    # Географический охват (ключевые слова, примеры)
    geo_keywords = ["мир", "мира", "россия", "европа", "страна", "регион", "америка", "азия", "африка"]
    geo = extract_keywords(description, geo_keywords)
    features["географический охват"] = geo if geo else None

    # Размер
    size = extract_size(description)
    features["размер"] = size

    # Масштаб
    scale = extract_scale(description)
    features["масштаб"] = scale

    # Тип карты: физическая, политическая, административная, историческая, транспортная и т.п.
    type_keywords = ["физическая", "политическая", "административная", "историческая", "транспортная", "дорожная", "топографическая", "контурная"]
    typ = extract_keywords(description, type_keywords)
    features["тип карты"] = typ if typ else None

    # Целевая аудитория
    audience_keywords = ["детская", "для детей", "школьная", "для школьников", "офисная"]
    audience = extract_keywords(description, audience_keywords)
    features["целевая аудитория"] = audience if audience else None

    # Язык
    language_keywords = ["русский", "английский", "билингв", "двуязычная"]
    lang = extract_keywords(description, language_keywords)
    features["язык"] = lang if lang else None

    # Стиль оформления
    style_keywords = ["винтаж", "ретро", "современная", "декоративная", "минималистическая"]
    style = extract_keywords(description, style_keywords)
    features["стиль оформления"] = style if style else None

    # Актуальность данных: пытаемся извлечь год; если есть слово "обновлен", то отмечаем как обновлённая
    year = extract_year(description)
    if "обновлен" in description.lower():
        features["актуальность"] = year if year else "обновленная"
    else:
        features["актуальность"] = year

    # Интерактивность и игровой формат
    interactive_keywords = ["интерактивная", "пазл", "скретч-карта", "игровая"]
    interactive = extract_keywords(description, interactive_keywords)
    features["интерактивность"] = interactive if interactive else None

    # Ламинация и специальные покрытия
    lam_keywords = ["ламинированная", "ламинация", "покрытие", "скретч", "флуоресцентное"]
    lam = extract_keywords(description, lam_keywords)
    features["ламинат/покрытие"] = lam if lam else None

    # Формат изделия (настенная, складная, атлас, глобус, плакат)
    format_keywords = ["настенная", "складная", "атлас", "глобус", "плакат"]
    form = extract_keywords(description, format_keywords)
    features["формат изделия"] = form if form else None

    # Способ крепления и материал
    mount_keywords = ["на холсте", "рейки", "магнит", "деревянная"]
    mount = extract_keywords(description, mount_keywords)
    features["способ крепления"] = mount if mount else None

    # Дополнительные элементы
    extra_keywords = ["флаг", "столица", "рельеф", "контур"]
    extra = extract_keywords(description, extra_keywords)
    features["дополнительные элементы"] = extra if extra else None

    return features

# Функция для сравнения характеристик исходного товара и целевого
def compare_features(source_features, target_features):
    similarity = {}
    # Для каждой характеристики рассчитываем меру схожести
    for key in source_features:
        src_val = source_features[key]
        tgt_val = target_features.get(key)
        
        if key == "размер":
            # Если оба размера есть, вычисляем относительную разницу для ширины и высоты
            if src_val and tgt_val:
                src_width, src_height = src_val
                tgt_width, tgt_height = tgt_val
                diff_width = abs(src_width - tgt_width) / src_width if src_width != 0 else 0
                diff_height = abs(src_height - tgt_height) / src_height if src_height != 0 else 0
                sim_width = 1 if diff_width < 0.1 else (0.5 if diff_width < 0.2 else 0)
                sim_height = 1 if diff_height < 0.1 else (0.5 if diff_height < 0.2 else 0)
                similarity[key] = (sim_width + sim_height) / 2
            else:
                similarity[key] = 0

        elif key == "масштаб":
            if src_val and tgt_val:
                diff = abs(src_val - tgt_val) / src_val if src_val != 0 else 0
                similarity[key] = 1 if diff < 0.01 else (0.5 if diff < 0.1 else 0)
            else:
                similarity[key] = 0

        elif key == "актуальность":
            if src_val and tgt_val:
                if isinstance(src_val, int) and isinstance(tgt_val, int):
                    diff = abs(src_val - tgt_val)
                    similarity[key] = 1 if diff == 0 else (0.5 if diff == 1 else 0)
                else:
                    similarity[key] = 1 if str(src_val).lower() == str(tgt_val).lower() else 0
            else:
                similarity[key] = 0

        else:
            # Для остальных категорий (ключевые слова)
            if src_val and tgt_val:
                similarity[key] = 1 if set(src_val) & set(tgt_val) else 0
            else:
                similarity[key] = 0

    return similarity

# Задаём веса для каждой характеристики
weights = {
    "географический охват": 3,    # высокая важность
    "размер": 2,                 # средняя важность
    "масштаб": 2,                # средняя важность
    "тип карты": 3,              # высокая важность
    "целевая аудитория": 3,      # высокая важность
    "язык": 1,                 # низкая важность
    "стиль оформления": 2,      # средняя важность
    "актуальность": 2,          # средняя важность
    "интерактивность": 3,       # высокая важность
    "ламинат/покрытие": 1,       # низкая важность
    "формат изделия": 2,         # средняя важность
    "способ крепления": 1,       # низкая важность
    "дополнительные элементы": 1 # низкая важность
}

# Функция для вычисления суммарного взвешенного показателя схожести
def compute_weighted_score(similarity_dict, weights):
    total = 0
    for key, weight in weights.items():
        total += similarity_dict.get(key, 0) * weight
    return total

# Функция для вычисления таблицы схожести для одного исходного товара
def compute_similarity_for_source(df, source_index, weights, num=10):
    # Извлекаем характеристики для всех товаров
    features_list = df["Name"].apply(extract_features)
    source_features = features_list.iloc[source_index]
    
    records = []
    for i, feat in features_list.items():
        if i == source_index:
            continue  # исключаем сравнение товара с самим собой
        sim_dict = compare_features(source_features, feat)
        weighted_score = compute_weighted_score(sim_dict, weights)
        record = {"product_index": i,
                  "SKU": df.loc[i, "SKU"],
                  "weighted_score": weighted_score}
        # Добавляем отдельные оценки по характеристикам (при необходимости)
        for key, value in sim_dict.items():
            record[key] = value
        records.append(record)
        
    result_df = pd.DataFrame(records)
    # Сортируем по суммарной взвешенной схожести (от наибольшего к наименьшему)
    result_df = result_df.sort_values(by="weighted_score", ascending=False)
    top10 = result_df.head(num)
    return top10

# Основной блок: обрабатываем исходные товары с индексами 0, 15 и 45
if __name__ == "__main__":
    # Чтение CSV-файла с описаниями товаров и SKU
    df = pd.read_csv("maps.csv")
    t = df[df["Seller"]=="ИНТЕРТРЕЙД"]
    source_indices = t.index
    #df = pd.read_csv("prodact.csv")
    df = df[["SKU","Name"]]
    
    
    final_df = pd.DataFrame()
    
    for src in source_indices:
        top10_df = compute_similarity_for_source(df, src, weights)
        top10_df["source_index"] = src  # индекс исходного товара
        source_sku = df.loc[src, "SKU"]
        top10_df["source_SKU"] = source_sku  # SKU исходного товара
        final_df = pd.concat([final_df, top10_df], ignore_index=True)
    
    
    # Сохраняем результат в CSV
    final_df.to_csv("top10_similarity_by_source.csv", index=False)


In [15]:
final_df[['source_SKU','SKU','weighted_score']]

,source_SKU,SKU,weighted_score
0,491279127,922231521,13.0
1,491279127,818165314,12.5
2,491279127,494562010,12.0
3,491279127,922230517,12.0
4,491279127,830206506,12.0
...,...,...,...
225,1200553001,600778837,10.0
226,1200553001,615607429,10.0
227,1200553001,1705141936,10.0
228,1200553001,950452977,10.0
